# Forecasting Infectious Disease Trends & Clustering Healthcare Utilization

This notebook will guide you step-by-step through forecasting COVID-19 trends and clustering healthcare utilization patterns across countries.

**Goal:**
- Predict future infection and healthcare demand trends.
- Group countries by similar healthcare usage patterns.

*All code and explanations are designed for beginners!*

## 1. Setup

Let's import the libraries we'll need for data analysis, modeling, and visualization.

In [ ]:
import pandas as pd  # For data handling
import numpy as np  # For numerical operations
import matplotlib.pyplot as plt  # For plotting
import seaborn as sns  # For prettier plots
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
import warnings
warnings.filterwarnings('ignore')  # Hide warnings for cleaner output

# Set plot style
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

## 2. Load and Explore Data

Let's load our cleaned datasets and take a look at what we're working with.

In [ ]:
# Load the cleaned datasets
train_df = pd.read_csv("../data/cleaned_train_data.csv")
test_df = pd.read_csv("../data/cleaned_test_data.csv")

# Convert date to datetime
train_df['date'] = pd.to_datetime(train_df['date'])
test_df['date'] = pd.to_datetime(test_df['date'])

# Display basic information about the datasets
print(f"Training Data Shape: {train_df.shape}")
print(f"Test Data Shape: {test_df.shape}")
print(f"Train date range: {train_df['date'].min()} to {train_df['date'].max()}")
print(f"Test date range: {test_df['date'].min()} to {test_df['date'].max()}")

In [ ]:
# Display the first few rows of the training data
train_df.head()

In [ ]:
# Check data types
print("Data Types:")
train_df.dtypes

In [ ]:
# Summary statistics for numerical columns
train_df.describe()

## 3. Data Preprocessing

Before we can build our models, we need to prepare the data. This includes:
1. Handling missing values
2. Feature engineering
3. Encoding categorical variables
4. Scaling numerical features

In [ ]:
# Check for missing values
missing_values = train_df.isnull().sum()
missing_values = missing_values[missing_values > 0]
print("Missing values in dataset:")
print(missing_values.sort_values(ascending=False))

In [ ]:
# Handle missing values
# For numerical columns, we'll use median imputation
numerical_cols = train_df.select_dtypes(include=['float64', 'int64']).columns
for col in numerical_cols:
    if col in missing_values.index:
        train_df[col].fillna(train_df[col].median(), inplace=True)
        test_df[col].fillna(train_df[col].median(), inplace=True)

# For categorical columns, we'll use mode imputation
categorical_cols = train_df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    if col in missing_values.index:
        train_df[col].fillna(train_df[col].mode()[0], inplace=True)
        test_df[col].fillna(train_df[col].mode()[0], inplace=True)

print("Missing values after imputation:")
print(train_df.isnull().sum().sum())

In [ ]:
# Feature engineering
# Extract temporal features from date
for df in [train_df, test_df]:
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    
    # Create lagged features for time series
    df['new_cases_lag1'] = df.groupby('location')['new_cases'].shift(1)
    df['new_cases_lag7'] = df.groupby('location')['new_cases'].shift(7)
    df['new_cases_lag14'] = df.groupby('location')['new_cases'].shift(14)
    
    # Create rolling statistics
    df['new_cases_rolling_mean7'] = df.groupby('location')['new_cases'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())
    df['new_cases_rolling_std7'] = df.groupby('location')['new_cases'].transform(lambda x: x.rolling(window=7, min_periods=1).std())
    
    # Create healthcare utilization metric
    if 'hospital_beds_per_thousand' in df.columns and 'total_cases' in df.columns:
        df['cases_per_bed'] = df['total_cases'] / (df['hospital_beds_per_thousand'] * df['population'] / 1000)
        df['healthcare_utilization'] = df['cases_per_bed'].fillna(0)

# Display the new features
new_features = ['year', 'month', 'day', 'dayofweek', 'quarter', 
                'new_cases_lag1', 'new_cases_lag7', 'new_cases_lag14',
                'new_cases_rolling_mean7', 'new_cases_rolling_std7',
                'cases_per_bed', 'healthcare_utilization']
train_df[new_features].head()

In [ ]:
# Encode categorical variables
categorical_cols = ['continent', 'location', 'tests_units']
label_encoders = {}

for col in categorical_cols:
    if col in train_df.columns:
        label_encoders[col] = LabelEncoder()
        train_df[col + '_encoded'] = label_encoders[col].fit_transform(train_df[col].astype(str))
        test_df[col + '_encoded'] = label_encoders[col].transform(test_df[col].astype(str))

# Scale numerical features
numerical_cols = train_df.select_dtypes(include=['float64', 'int64']).columns
numerical_cols = [col for col in numerical_cols if col not in ['Id'] and not col.endswith('_encoded')]

scaler = StandardScaler()
train_df[numerical_cols] = scaler.fit_transform(train_df[numerical_cols])
test_df[numerical_cols] = scaler.transform(test_df[numerical_cols])

print("Data preprocessing complete!")

## 4. Time Series Forecasting

Now that our data is prepared, let's build models to forecast future COVID-19 trends. We'll focus on predicting:
1. New cases per day
2. New deaths per day
3. Healthcare utilization

In [ ]:
# Define target variables
target_variables = ['new_cases', 'new_deaths', 'healthcare_utilization']

# Define features to use for prediction
feature_cols = [col for col in train_df.columns if col not in ['Id', 'date', 'iso_code', 'location', 'continent', 'tests_units'] 
                and not col.endswith('_encoded') and col not in target_variables]

print(f"Number of features: {len(feature_cols)}")
print(f"Features: {feature_cols[:10]}...")

In [ ]:
# Split data for time series forecasting
# We'll use a time-based split to maintain temporal integrity
train_cutoff_date = train_df['date'].quantile(0.8)
train_forecast = train_df[train_df['date'] < train_cutoff_date].copy()
val_forecast = train_df[train_df['date'] >= train_cutoff_date].copy()

print(f"Training data for forecasting: {train_forecast.shape[0]} rows")
print(f"Validation data for forecasting: {val_forecast.shape[0]} rows")

In [ ]:
# Define models for forecasting
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'XGBoost': xgb.XGBRegressor(n_estimators=100, random_state=42),
    'LightGBM': lgb.LGBMRegressor(n_estimators=100, random_state=42),
    'CatBoost': CatBoostRegressor(iterations=100, random_state=42, verbose=0)
}

# Function to evaluate models
def evaluate_model(model, X_train, y_train, X_val, y_val, model_name):
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_val)
    
    # Calculate metrics
    mse = mean_squared_error(y_val, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_val, y_pred)
    r2 = r2_score(y_val, y_pred)
    
    print(f"{model_name} - RMSE: {rmse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")
    
    return model, rmse

In [ ]:
# Train and evaluate models for each target variable
best_models = {}
best_scores = {}

for target in target_variables:
    print(f"\nForecasting {target}:")
    
    # Prepare data
    X_train = train_forecast[feature_cols].fillna(0)
    y_train = train_forecast[target].fillna(0)
    X_val = val_forecast[feature_cols].fillna(0)
    y_val = val_forecast[target].fillna(0)
    
    # Train and evaluate each model
    best_rmse = float('inf')
    best_model = None
    
    for name, model in models.items():
        trained_model, rmse = evaluate_model(model, X_train, y_train, X_val, y_val, name)
        
        if rmse < best_rmse:
            best_rmse = rmse
            best_model = trained_model
    
    best_models[target] = best_model
    best_scores[target] = best_rmse
    
    print(f"Best model for {target}: {best_model.__class__.__name__} with RMSE: {best_rmse:.4f}")

In [ ]:
# Visualize predictions for the best model for new_cases
if 'new_cases' in best_models:
    # Get predictions
    X_val = val_forecast[feature_cols].fillna(0)
    y_val = val_forecast['new_cases'].fillna(0)
    y_pred = best_models['new_cases'].predict(X_val)
    
    # Plot actual vs predicted
    plt.figure(figsize=(14, 6))
    plt.plot(val_forecast['date'], y_val, label='Actual')
    plt.plot(val_forecast['date'], y_pred, label='Predicted')
    plt.title('New Cases: Actual vs Predicted')
    plt.xlabel('Date')
    plt.ylabel('New Cases')
    plt.legend()
    plt.tight_layout()
    plt.show()

## 5. Clustering Healthcare Utilization

Now, let's cluster countries based on their healthcare utilization patterns. This will help us identify groups of countries with similar healthcare system responses to COVID-19.

In [ ]:
# Prepare data for clustering
# We'll use the latest data for each country
latest_data = train_df.sort_values('date').groupby('location').last().reset_index()

# Select healthcare-related features
healthcare_features = [
    'hospital_beds_per_thousand',
    'life_expectancy',
    'human_development_index',
    'healthcare_utilization',
    'total_cases_per_million',
    'total_deaths_per_million'
]

# Drop rows with missing values
healthcare_data = latest_data[['location'] + healthcare_features].dropna()

print(f"Number of countries for clustering: {len(healthcare_data)}")

In [ ]:
# Scale the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(healthcare_data[healthcare_features])

# Apply PCA for dimensionality reduction and visualization
pca = PCA(n_components=2)
pca_result = pca.fit_transform(scaled_data)

# Create a DataFrame with PCA results
pca_df = pd.DataFrame(data=pca_result, columns=['PC1', 'PC2'])
pca_df['location'] = healthcare_data['location'].values

# Plot PCA results
plt.figure(figsize=(12, 8))
plt.scatter(pca_df['PC1'], pca_df['PC2'])
plt.title('PCA of Healthcare Features')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')

# Add country labels to some points
for i, country in enumerate(pca_df['location']):
    if i % 5 == 0:  # Label every 5th country to avoid overcrowding
        plt.annotate(country, (pca_df['PC1'][i], pca_df['PC2'][i]))

plt.tight_layout()
plt.show()

In [ ]:
# Determine optimal number of clusters using silhouette score
from sklearn.metrics import silhouette_score

silhouette_scores = []
k_range = range(2, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    cluster_labels = kmeans.fit_predict(scaled_data)
    silhouette_avg = silhouette_score(scaled_data, cluster_labels)
    silhouette_scores.append(silhouette_avg)
    print(f"For n_clusters = {k}, the average silhouette score is: {silhouette_avg:.4f}")

# Plot silhouette scores
plt.figure(figsize=(10, 6))
plt.plot(k_range, silhouette_scores, marker='o')
plt.title('Silhouette Score vs. Number of Clusters')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.grid(True)
plt.show()

In [ ]:
# Apply K-means clustering with the optimal number of clusters
optimal_k = k_range[np.argmax(silhouette_scores)]
print(f"Optimal number of clusters: {optimal_k}")

kmeans = KMeans(n_clusters=optimal_k, random_state=42)
healthcare_data['Cluster'] = kmeans.fit_predict(scaled_data)

# Visualize clusters in PCA space
pca_df['Cluster'] = healthcare_data['Cluster'].values

plt.figure(figsize=(12, 8))
scatter = plt.scatter(pca_df['PC1'], pca_df['PC2'], c=pca_df['Cluster'], cmap='viridis')
plt.title('Country Clusters Based on Healthcare Metrics')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.colorbar(scatter, label='Cluster')

# Add country labels to some points
for i, country in enumerate(pca_df['location']):
    if i % 5 == 0:  # Label every 5th country to avoid overcrowding
        plt.annotate(country, (pca_df['PC1'][i], pca_df['PC2'][i]))

plt.tight_layout()
plt.show()

In [ ]:
# Analyze cluster characteristics
cluster_analysis = healthcare_data.groupby('Cluster')[healthcare_features].mean()
print("Cluster Characteristics:")
cluster_analysis

In [ ]:
# Display countries in each cluster
for cluster in range(optimal_k):
    print(f"\nCluster {cluster}:")
    countries = healthcare_data[healthcare_data['Cluster'] == cluster]['location'].tolist()
    print(countries)

## 6. Conclusion and Insights

Let's summarize our findings and provide actionable insights based on our analysis.

In [ ]:
# Print summary of best forecasting models
print("Best Forecasting Models:")
for target, model in best_models.items():
    print(f"- {target}: {model.__class__.__name__} with RMSE: {best_scores[target]:.4f}")

In [ ]:
# Print summary of clustering results
print("\nClustering Results:")
print(f"- Optimal number of clusters: {optimal_k}")
print("\nCluster Characteristics:")
for cluster in range(optimal_k):
    countries = healthcare_data[healthcare_data['Cluster'] == cluster]['location'].tolist()
    print(f"\nCluster {cluster} ({len(countries)} countries):")
    print(f"Countries: {', '.join(countries[:5])}{'...' if len(countries) > 5 else ''}")
    
    # Print key characteristics
    cluster_data = healthcare_data[healthcare_data['Cluster'] == cluster]
    print(f"Average healthcare utilization: {cluster_data['healthcare_utilization'].mean():.4f}")
    print(f"Average hospital beds per thousand: {cluster_data['hospital_beds_per_thousand'].mean():.4f}")
    print(f"Average human development index: {cluster_data['human_development_index'].mean():.4f}")

### Key Insights:

1. **Forecasting Insights**:
   - [Your insights will appear here after running the notebook]

2. **Clustering Insights**:
   - [Your insights will appear here after running the notebook]

3. **Recommendations for Healthcare Systems**:
   - [Your recommendations will appear here after running the notebook]